In [1]:
import torch
from torch.utils.data import Dataset, DataLoader
import pandas as pd
import numpy as np

data = pd.read_csv("/kaggle/input/suduko-night-shift/trained_output.csv")
data

,Concatenated Values,Concatenated Values.1
0,0636554170018000590950008620700009000000700203...,6832954174218673597953418625721349968465771233...
1,1007540320002060070048390500010780000405903100...,1697548323582169472748396515314782967465923189...
2,7000508398000250409300782002798004000004970266...,7126548398659237419341782652798614533584971266...
3,0000930078030000290590824039003502016001007081...,2145936878637415297596824139783542616351297481...
4,6507300901986000070370002560003456020000010762...,6527348912986524374371982569713456823852619742...
...,...,...
49995,0896000576200007000408102060107098239020800000...,1896243576279351845438172964167598239724835613...
49996,7413090080050800402300000701000360276570481903...,7413592689657823412386149751895364276572451933...
49997,2004639101040080000800057034200070060008064919...,2574639181347986256892157434219573863758264919...
49998,9082000000024609706748000507000193800000025003...,9382571461524639786748912537256193844693825173...


In [2]:
class SudokuDataset(Dataset):
    def __init__(self, dataframe, subset="train"):
        self.dataframe = dataframe
        self.subset = subset

    def __len__(self):
        return len(self.dataframe)

    def __getitem__(self, idx):
        puzzle = self.dataframe.iloc[idx, 0]
        solution = self.dataframe.iloc[idx, 1]

        puzzle = torch.tensor([int(p) for p in puzzle], dtype=torch.float32).reshape(1, 9, 9) / 9 - 0.5
        if self.subset == 'train':
            # Subtracting 1 from each solution value to match PyTorch's 0-based indexing
            solution = torch.tensor([int(s) - 1 for s in solution], dtype=torch.int64).reshape(81)
            return puzzle, solution
        return puzzle

In [3]:
train_idx = int(len(data) * 0.75)
train_data = SudokuDataset(data.iloc[:train_idx], subset="train")
train_loader = DataLoader(train_data, batch_size=640, shuffle=True)

val_data = SudokuDataset(data.iloc[train_idx:], subset="train")
val_loader = DataLoader(val_data, batch_size=640, shuffle=False)

In [4]:
import torch.nn as nn
import torch.nn.functional as F

class SudokuSolver(nn.Module):
    def __init__(self):
        super(SudokuSolver, self).__init__()
        self.conv1 = nn.Conv2d(1, 64, kernel_size=3, padding=1)
        self.bn1 = nn.BatchNorm2d(64)
        self.conv2 = nn.Conv2d(64, 128, kernel_size=3, padding=1)
        self.bn2 = nn.BatchNorm2d(128)
        self.conv3 = nn.Conv2d(128, 128, kernel_size=1)
        self.fc1 = nn.Linear(128 * 9 * 9, 81 * 9)

    def forward(self, x):
        x = F.relu(self.bn1(self.conv1(x)))
        x = F.relu(self.bn2(self.conv2(x)))
        x = F.relu(self.conv3(x))
        x = x.view(x.size(0), -1)
        x = self.fc1(x)
        return x.view(-1, 9)  # Reshaped to match [batch_size * 81, 9]

model = SudokuSolver()

In [5]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
criterion = nn.CrossEntropyLoss()

In [6]:
def calculate_accuracy(outputs, labels):
    _, predicted = torch.max(outputs, dim=1)
    correct = (predicted == labels).sum().item()
    return correct / labels.size(0)

def train_model(model, train_loader, val_loader, epochs):
    for epoch in range(epochs):
        model.train()
        train_loss = 0.0
        train_correct = 0
        total_train = 0
        # Training Phase
        for puzzles, solutions in train_loader:
            puzzles, solutions = puzzles.to(device), solutions.to(device)
            optimizer.zero_grad()
            outputs = model(puzzles)
            loss = criterion(outputs, solutions.view(-1))
            loss.backward()
            optimizer.step()

            train_loss += loss.item()
            train_correct += calculate_accuracy(outputs, solutions.view(-1)) * solutions.size(0)
            total_train += solutions.size(0)

        avg_train_loss = train_loss / len(train_loader)
        train_accuracy = train_correct / total_train
        # Validation Phase
        model.eval()
        val_loss = 0.0
        val_correct = 0
        total_val = 0

        with torch.no_grad():
            for puzzles, solutions in val_loader:
                puzzles, solutions = puzzles.to(device), solutions.to(device)
                outputs = model(puzzles)
                val_loss += criterion(outputs, solutions.view(-1)).item()

                val_correct += calculate_accuracy(outputs, solutions.view(-1)) * solutions.size(0)
                total_val += solutions.size(0)

        avg_val_loss = val_loss / len(val_loader)
        val_accuracy = val_correct / total_val

        print(f'Epoch {epoch + 1}: '
              f'Train Loss: {avg_train_loss:.4f}, '
              f'Train Accuracy: {train_accuracy:.4f}, '
              f'Validation Loss: {avg_val_loss:.4f}, '
              f'Validation Accuracy: {val_accuracy:.4f}')

In [7]:
train_model(model, train_loader, val_loader, 12)

Epoch 1: Train Loss: 2.1405, Train Accuracy: 0.1572, Validation Loss: 2.0089, Validation Accuracy: 0.1995
Epoch 2: Train Loss: 1.9049, Train Accuracy: 0.2563, Validation Loss: 1.8147, Validation Accuracy: 0.2985
Epoch 3: Train Loss: 1.5626, Train Accuracy: 0.4147, Validation Loss: 1.4413, Validation Accuracy: 0.4640
Epoch 4: Train Loss: 1.2048, Train Accuracy: 0.5811, Validation Loss: 1.1642, Validation Accuracy: 0.5992
Epoch 5: Train Loss: 0.9517, Train Accuracy: 0.6856, Validation Loss: 1.0021, Validation Accuracy: 0.6589
Epoch 6: Train Loss: 0.7846, Train Accuracy: 0.7421, Validation Loss: 0.9207, Validation Accuracy: 0.6850
Epoch 7: Train Loss: 0.6627, Train Accuracy: 0.7840, Validation Loss: 0.8706, Validation Accuracy: 0.7011
Epoch 8: Train Loss: 0.5668, Train Accuracy: 0.8178, Validation Loss: 0.8536, Validation Accuracy: 0.7090
Epoch 9: Train Loss: 0.4884, Train Accuracy: 0.8452, Validation Loss: 0.8564, Validation Accuracy: 0.7119
Epoch 10: Train Loss: 0.4224, Train Accuracy: 

## Testing

In [17]:
import pandas as pd
test=pd.read_csv("/kaggle/input/suduko-night-shift/Test concatenated_values.csv")
test

,Concatenated Values
0,9027008637614000250800000706070950009106000542...
1,0708010200207300001060920040900754800520080038...
2,0009001650806000306012437008070950230307046009...
3,1580096070000760027264183000157040000040800510...
4,0020500096038002059700008361080005032060350800...
...,...
9995,0001050300407062100009405607510943060202507040...
9996,1058000404001300056030509170006005315060030092...
9997,0000278000703000900489500367800000040624009783...
9998,0020950000391800001050020980017200803275009164...


In [20]:
def preprocess_puzzle(puzzle_str):
    """
    Preprocess the input Sudoku puzzle string into a format suitable for the model.
    """
    puzzle_tensor = torch.tensor([int(p) for p in puzzle_str], dtype=torch.float32).reshape(1, 1, 9, 9) / 9 - 0.5
    return puzzle_tensor.to(device)

def predict_test_set(model, test_df):
    """
    Predict solutions for all puzzles in the test DataFrame.
    """
    model.eval()
    predictions = []

    for _, row in test_df.iterrows():
        puzzle_str = row['Concatenated Values']
        with torch.no_grad():
            puzzle_tensor = preprocess_puzzle(puzzle_str)
            output = model(puzzle_tensor)
            solution = output.argmax(dim=1).view(9, 9) + 1  # Convert predictions to 1-based indexing
            predictions.append(solution.cpu().numpy())

    return predictions

In [21]:
# predict on the entire test set
predicted_solutions = predict_test_set(model, test)

In [22]:
indexes = []
labels= []
#for i,j in enumerate(predicted_solutions_df["predicted_solution"]):
for i,j in enumerate(predicted_solutions):
    preds = j.reshape(9,9)
    for row in range(0,9):
        for col in range(0,9):
            index = str(i)+"_"+str(row)+str(col)
            indexes.append(index)
            labels.append(preds[row,col])

In [23]:
Test_Predictions =  {"id":indexes , "values":labels}
Test_Predictions = pd.DataFrame(Test_Predictions)
Test_Predictions

,id,values
0,0_00,4
1,0_01,4
2,0_02,2
3,0_03,7
4,0_04,1
...,...,...
809995,9999_84,1
809996,9999_85,8
809997,9999_86,3
809998,9999_87,9


In [24]:
Test_Predictions.to_csv("Test_Predictions.csv", index=False)